In [1]:
import os
import pandas as pd 
import numpy as np
# from numpy.random import seed
import random

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization 
from tensorflow.keras import utils 
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.metrics import AUC

from sklearn.metrics import roc_auc_score, recall_score
from sklearn.model_selection import KFold
from hyperopt import tpe, hp, fmin, STATUS_OK, Trials
from hyperopt.pyll.base import scope
from hyperopt import space_eval

ModuleNotFoundError: No module named 'tensorflow'

## Подготовка данных

In [2]:
!wget https://lodmedia.hb.bizmrg.com/case_files/758269/train_dataset_train.zip
!wget https://lodmedia.hb.bizmrg.com/case_files/758269/test_dataset_test.csv
!md5sum train_dataset_train.zip test_dataset_test.csv

In [3]:
!unzip train_dataset_train.zip

In [4]:
data_df = pd.read_csv('train.csv')
# data_df

In [5]:
test_df = pd.read_csv('test_dataset_test.csv')
# test_df

In [6]:
data_df['Сигарет в день'] = data_df['Сигарет в день'].fillna(0)
test_df['Сигарет в день'] = test_df['Сигарет в день'].fillna(0)

In [7]:
# Создаём словарь поле - его индекс
def create_dict(s):
  ret = {}                          # Создаём пустой словарь
  for _id, name in enumerate(s):    # Проходим по всем парам - id и название
    ret.update({name: _id})         # Добавляем в словарь
  return ret

In [8]:
data_df.columns

In [9]:
mul_features = ['Пол', 'Семья', 'Этнос', #'Национальность', 'Религия',
       'Образование', 'Профессия',
       'Статус Курения',
       'Частота пасс кур', 'Алкоголь']
    #    'Время засыпания', 'Время пробуждения']
    #    'Возраст курения', 'Возраст алког',
bin_features = ['Вы работаете?', 'Выход на пенсию',
       'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
       'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма',
       'Туберкулез легких ', 'ВИЧ/СПИД',
       'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
       'Пассивное курение', 
       'Сон после обеда',
       'Спорт, клубы', 'Религия, клубы']

In [10]:
diseases = ['Артериальная гипертензия',
       'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда',
       'Сердечная недостаточность', 'Прочие заболевания сердца']

In [11]:
all_df = pd.concat([data_df, test_df])

In [12]:
dic = {}
for feature in mul_features:
    dic[feature] = create_dict(set(all_df[feature]))
dic

In [13]:
y_data = []
for i in range(5):
    y_data.append(data_df[diseases[i]].values)
    print(y_data[i].mean())

In [14]:
x_data = []

for _id, person in enumerate(np.array(data_df)):
    x_tr = []
    for feature in mul_features:
        arr = np.zeros(len(dic[feature]), np.float32)
        arr[dic[feature][data_df.loc[_id, feature]]] = 1.
        x_tr.extend(arr)

    for feature in bin_features:
        x_tr.append(data_df.loc[_id, feature])
    x_tr.append(data_df.loc[_id, 'Сигарет в день'] / 10.)
    x_data.append(x_tr)

x_data = np.array(x_data, dtype=np.float32)
x_train = x_data[:]
# x_val = x_data[N:]
y_train = []
# y_val = []
for i in range(5):
    y_train.append(y_data[i])
    # y_val.append(y_data[i][N:])

print(x_train.shape, y_train[0].shape) #, x_val.shape, y_val[4].shape)

In [15]:
x_test = []

for _id, person in enumerate(np.array(test_df)):
    x_tr = []
    # print(_id)
    for feature in mul_features:
        arr = np.zeros(len(dic[feature]), np.float64)
        
        # print(feature, test_df.loc[_id, feature])
        arr[dic[feature][test_df.loc[_id, feature]]] = 1.
        x_tr.extend(arr)

    for feature in bin_features:
        x_tr.append(test_df.loc[_id, feature])
    x_tr.append(test_df.loc[_id, 'Сигарет в день'] / 10.)
    x_test.append(x_tr)

x_test = np.array(x_test, dtype=np.float64)
x_test.shape

## Подбор гиперпараметров

In [16]:
def get_pred(probs, th=0.5):
    y_pred = np.zeros(len(probs), dtype=np.float32)
    for i, p in enumerate(probs):
        if p > th:
            y_pred[i] = 1
    return y_pred

def best_th(y_true, probs):
    best_th = 0.5
    best_recall = 0.
    for th in np.linspace(0, 1, 201):
        y_pred = get_pred(probs, th)
        recall = recall_score(y_true, y_pred, average='macro')
        if recall > best_recall:
            best_recall = recall
            best_th = th
    # print("th=%.3f recall=%.4f" % (best_th, best_recall))
    return best_th, best_recall

In [17]:
space = {
    # "batch_size": hp.choice('batch_size', list(range(8, 128))),
    # "act_fn": hp.choice('act_fn', ['sigmoid', 'tanh', 'elu', 'relu', 'relu6']),
    "opt": hp.choice('opt', [
        'adadelta',
        'adagrad',
        'adam',
        'ftlr',
        'nadam',
        'rmsprop',
        'sgd'
    ]),
    'num_layers': hp.choice('num_layers', [1, 2]),
    'num_epochs': 15 + hp.randint('num_epochs', 60),
    'lr': hp.uniform('lr', 1e-5, 1e-2),
    "latent_dim": hp.choice('latent_dim', [64, 128, 256, 512, 1024, 2048]),
    "latent_dim2": hp.choice('latent_dim2', [16, 32, 64, 128]),
    "dropout": hp.uniform('dropout', 0., 1.),
}

In [18]:
def build_model(config):
    model = Sequential()

    model.add(Dense(config['latent_dim'], activation='relu'))
    # if config['batch_norm'] > 0:
    #     model.add(BatchNormalization())
    model.add(Dropout(config['dropout']))

    if config['num_layers'] > 1:
        model.add(Dense(config['latent_dim2'], activation='relu'))
        # if config['batch_norm'] > 0:
        #     model.add(BatchNormalization())
        model.add(Dropout(config['dropout']))

    model.add(Dense(1, activation='sigmoid'))

    if config['opt'] == 'adadelta':
      opt = tf.keras.optimizers.Adadelta(learning_rate=config['lr'])
    if config['opt'] == 'adagrad':
      opt = tf.keras.optimizers.Adagrad(learning_rate=config['lr'])
    if config['opt'] == 'adam':
      opt = tf.keras.optimizers.Adam(learning_rate=config['lr'])
    if config['opt'] == 'ftlr':
      opt = tf.keras.optimizers.Ftrl(learning_rate=config['lr'])
    if config['opt'] == 'nadam':
      opt = tf.keras.optimizers.Nadam(learning_rate=config['lr'])
    if config['opt'] == 'rmsprop':
      opt = tf.keras.optimizers.RMSprop(learning_rate=config['lr'])
    if config['opt'] == 'sgd':
      opt = tf.keras.optimizers.SGD(learning_rate=config['lr'])
    model.compile(
        optimizer=opt, 
        loss='binary_crossentropy', 
        metrics=[
        #   tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        #   tf.keras.metrics.Recall(name='recall'),
        #   tf.keras.metrics.AUC(name='auc')
        ]
    )

    return model

In [34]:
def cross_validation(config, X, y, num_folds, num_epochs):
    cv_scores = []
    ths = []
    seed = 101
    kf = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    for train_index, val_index in kf.split(X):
        x_train = x_data[train_index]
        y_train = y[train_index]
        x_val = x_data[val_index]
        y_val = y[val_index]
        tf.random.set_seed(seed)
        model = build_model(config)
        hist = model.fit(
            x_train, y_train,
            epochs = num_epochs,
            batch_size=len(x_train),
            # validation_data=(x_val, y_val),
            verbose=0
        )
        val_probs = model.predict(x_val)
        th, recall = best_th(y_val, val_probs)
        # print("Recall=%.4f th=%.3f" % (recall, th))
        cv_scores.append(recall)
        ths.append(th)
    return cv_scores, ths

In [35]:
K_FOLDS = 5
def objective(params):
    d = params['d']
    print(f"d={d} Selected model hyperparams:")
    print(params)
    cv, ths = cross_validation(
        config=params,
        X=x_data,
        y=y_data[d],
        num_folds=K_FOLDS,
        num_epochs=params['num_epochs']
    )
    mean_cv = np.mean(cv)
    std_cv = np.std(cv)
    mean_th = np.mean(ths)
    print('Avg CV score: %.4f Avg th: %.4f' % (mean_cv, mean_th))
    return {'loss': -mean_cv, 'status': STATUS_OK, 'th': mean_th, 'recall': mean_cv, 'recall_std': std_cv, 'params': params}

In [36]:
%%time
N_EVALS = 200
best_configs = [None] * 5
all_trials = [None] * 5
for d in range(5):
    space['d'] = d
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space, 
        algo=tpe.suggest, 
        max_evals=N_EVALS, 
        trials=trials
    )
    all_trials[d] = trials
    best_config = space_eval(space, best)
    best_configs[d] = best_config
best_configs

```
[{'d': 0,
  'dropout': 0.5734774403761633,
  'latent_dim': 256,
  'latent_dim2': 64,
  'lr': 0.0016233818518730145,
  'num_epochs': 56,
  'num_layers': 1,
  'opt': 'adam'},
 {'d': 1,
  'dropout': 0.450068425945272,
  'latent_dim': 512,
  'latent_dim2': 128,
  'lr': 0.006648389773229169,
  'num_epochs': 44,
  'num_layers': 1,
  'opt': 'adam'},
 {'d': 2,
  'dropout': 0.9464760296798049,
  'latent_dim': 128,
  'latent_dim2': 64,
  'lr': 0.008969297837338628,
  'num_epochs': 40,
  'num_layers': 1,
  'opt': 'nadam'},
 {'d': 3,
  'dropout': 0.822267450844782,
  'latent_dim': 64,
  'latent_dim2': 16,
  'lr': 0.007205635676538558,
  'num_epochs': 38,
  'num_layers': 1,
  'opt': 'nadam'},
 {'d': 4,
  'dropout': 0.024777900663576657,
  'latent_dim': 64,
  'latent_dim2': 32,
  'lr': 0.00704502498414389,
  'num_epochs': 71,
  'num_layers': 1,
  'opt': 'rmsprop'}]
  ```

In [37]:
all_best_params = [None] * 5
for d in range(5):
    best_recall = 0
    for res in all_trials[d].results:
        recall = res['recall']
        if recall > best_recall:
            best_recall = recall
            best_params = res['params']
            best_params['recall'] = recall
            best_params['recall_std'] = res['recall_std']
            best_params['th'] = res['th']
    all_best_params[d] = best_params
all_best_params

N=100 seed=42
```
[{'d': 0,
  'dropout': 0.5734774403761633,
  'latent_dim': 256,
  'latent_dim2': 64,
  'lr': 0.0016233818518730145,
  'num_epochs': 56,
  'num_layers': 1,
  'opt': 'adam',
  'recall': 0.741370765107158,
  'recall_std': 0.014620953740119883,
  'th': 0.43900000000000006},
 {'d': 1,
  'dropout': 0.450068425945272,
  'latent_dim': 512,
  'latent_dim2': 128,
  'lr': 0.006648389773229169,
  'num_epochs': 44,
  'num_layers': 1,
  'opt': 'adam',
  'recall': 0.740704607382484,
  'recall_std': 0.10955485937401051,
  'th': 0.062},
 {'d': 2,
  'dropout': 0.9464760296798049,
  'latent_dim': 128,
  'latent_dim2': 64,
  'lr': 0.008969297837338628,
  'num_epochs': 40,
  'num_layers': 1,
  'opt': 'nadam',
  'recall': 0.7353592991935203,
  'recall_std': 0.030500378684814082,
  'th': 0.134},
 {'d': 3,
  'dropout': 0.822267450844782,
  'latent_dim': 64,
  'latent_dim2': 16,
  'lr': 0.007205635676538558,
  'num_epochs': 38,
  'num_layers': 1,
  'opt': 'nadam',
  'recall': 0.720422176874554,
  'recall_std': 0.055569887230576605,
  'th': 0.085},
 {'d': 4,
  'dropout': 0.024777900663576657,
  'latent_dim': 64,
  'latent_dim2': 32,
  'lr': 0.00704502498414389,
  'num_epochs': 71,
  'num_layers': 1,
  'opt': 'rmsprop',
  'recall': 0.632316767963341,
  'recall_std': 0.09494634984993429,
  'th': 0.067}]
```

N=200 seed=42
```
[{'d': 0,
  'dropout': 0.14945121897024244,
  'latent_dim': 128,
  'latent_dim2': 128,
  'lr': 0.0009747128348755505,
  'num_epochs': 43,
  'num_layers': 2,
  'opt': 'rmsprop',
  'recall': 0.7404515741709105,
  'recall_std': 0.018458233956510448,
  'th': 0.4640000000000001},
 {'d': 1,
  'dropout': 0.7691112910491312,
  'latent_dim': 64,
  'latent_dim2': 128,
  'lr': 0.006500364050753514,
  'num_epochs': 34,
  'num_layers': 2,
  'opt': 'nadam',
  'recall': 0.6960120169584498,
  'recall_std': 0.09074418483503259,
  'th': 0.027000000000000003},
 {'d': 2,
  'dropout': 0.22627320459511344,
  'latent_dim': 256,
  'latent_dim2': 64,
  'lr': 0.0008823890513231178,
  'num_epochs': 61,
  'num_layers': 2,
  'opt': 'adam',
  'recall': 0.7355128594367655,
  'recall_std': 0.040440097374479296,
  'th': 0.10200000000000001},
 {'d': 3,
  'dropout': 0.5731037441915432,
  'latent_dim': 2048,
  'latent_dim2': 128,
  'lr': 0.00920552135498809,
  'num_epochs': 27,
  'num_layers': 1,
  'opt': 'adam',
  'recall': 0.7171118778654642,
  'recall_std': 0.032206004429925175,
  'th': 0.12000000000000002},
 {'d': 4,
  'dropout': 0.7758672877829247,
  'latent_dim': 256,
  'latent_dim2': 64,
  'lr': 0.0075772436760977065,
  'num_epochs': 25,
  'num_layers': 2,
  'opt': 'nadam',
  'recall': 0.6195179464738917,
  'recall_std': 0.0279690037640686,
  'th': 0.052000000000000005}]
  ```
  

In [38]:
avg_recall = 0.
var = 0.
for d in range(5):
    recall = all_best_params[d]['recall']
    std = all_best_params[d]['recall_std']
    var += std*std
    print("d=%d recall=%.4f" % (d, recall))
    avg_recall += recall
avg_recall /= 5
var /= 5
std = pow(var, 0.5)
print('Avg recall=%.4f std=%.4f' % (avg_recall, std))

```
d=0 recall=0.7414
d=1 recall=0.7407
d=2 recall=0.7354
d=3 recall=0.7204
d=4 recall=0.6323
Avg recall=0.7140 std=0.0711
```

## Обучение моделей


In [47]:
%%time
models = []
seed = 137
for d in range(5):
    params = all_best_params[d]
    num_epochs = params['num_epochs']
    tf.random.set_seed(seed)
    model = build_model(params)
    history = model.fit(x_data, y_data[d], batch_size=len(x_data), epochs=num_epochs, verbose=0)
    models.append(model)
    

## Предсказание на тесте

In [48]:
ths = [0.5]*5
y_preds = [None] * 5
avg_recall = 0.
for d in range(5):
    test_probs = models[d].predict(x_test)
    recall = all_best_params[d]['recall']
    th = all_best_params[d]['th']
    y_preds[d] = get_pred(test_probs, th)
    print("d=%d th=%.3f Recall=%.4f" % (d, th, recall))
    avg_recall += recall
avg_recall /= 5
print("Avg Recall=%.4f" % avg_recall)

In [49]:
res_df = pd.DataFrame()
res_df['ID'] = test_df.ID.values
for d in range(5):
    res_df[diseases[d]] = y_preds[d]
res_df


In [50]:
name = 'kaggle-%4d-%d.csv' % (int(avg_recall * 1e4), seed)
print(name)
res_df.to_csv(name, index=False)

## Прочее

In [ ]:
%%time
d = 4
num_epochs = best_configs[d]['num_epochs']
recalls = []
ths = []
for i in range(10):
    seed = i + 300
    tf.random.set_seed(seed)
    model = build_model(best_configs[d])
    history = model.fit(x_train, y_train[d], batch_size=len(x_train), epochs=num_epochs, validation_data=(x_val, y_val[d]), verbose=0)
    val_probs = model.predict(x_val)
    th, recall = best_th(y_val[d], val_probs)
    recalls.append(recall)
    ths.append(th)
    print('.', end='')
print()
recalls = np.array(recalls)
ths = np.array(ths)
print("Recalls: mean=%.4f mse=%.4f max=%.4f min=%.4f" % (recalls.mean(), recalls.std(), recalls.max(), recalls.min()))
      

In [ ]:
%%time
for d in range(5):
    seed = i + seeds[d]
    tf.random.set_seed(seed)
    model = build_model(best_configs[d])
    history = model.fit(x_train, y_train[d], batch_size=len(x_train), epochs=ep[d], validation_data=(x_val, y_val[d]), verbose=0)
    val_probs = model.predict(x_val)
    th, recall = best_th(y_val[d], val_probs)
    if recall > recalls[d]:
        recalls[d] = recall
        models[d] = model
        seeds[d] = seed
        print("Updated, seed=%d recall=%.4f (d=%d)" % (seed, recall, d))
    print("d=%d Best Recall %.4f" % (d, recalls[d]))    


## model 1

In [ ]:
def build_model(th=0.5):
    model = Sequential()

    model.add(Dense(1024, activation='relu'))
    # model.add(BatchNormalization())
    model.add(Dropout(0.8))

    model.add(Dense(64, activation='relu'))
    # model.add(BatchNormalization())
    model.add(Dropout(0.8))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy', 
        optimizer=Adam(learning_rate=0.001),
        metrics=[
          'accuracy',
        #   tf.keras.metrics.Recall(thresholds=th),
        #   tf.keras.metrics.AUC()
        ]
    )

    return model


In [ ]:
def plot(history):
    plt.plot(history.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
    plt.plot(history.history['val_accuracy'], 
            label='Доля верных ответов на проверочном наборе')
    plt.xlabel('Эпоха обучения')
    plt.ylabel('Доля верных ответов')
    plt.legend()
    plt.grid()
    plt.show()

    # plt.plot(history.history['recall'], 
    #      label='Recall на обучающем наборе')
    # plt.plot(history.history['val_recall'], 
    #         label='Recall на проверочном наборе')
    # plt.xlabel('Эпоха обучения')
    # plt.ylabel('Recall')
    # plt.legend()
    # plt.grid()
    # plt.show()

    # plt.plot(history.history['auc_2'], 
    #      label='AUC на обучающем наборе')
    # plt.plot(history.history['val_auc_2'], 
    #         label='AUC на проверочном наборе')
    # plt.xlabel('Эпоха обучения')
    # plt.ylabel('AUC')
    # plt.legend()
    # plt.grid()
    # plt.show()

    # Выводим графики ошибки
    plt.plot(history.history['loss'], 
            label='Ошибка на обучающем наборе')
    plt.plot(history.history['val_loss'], 
            label='Ошибка на проверочном наборе')
    plt.xlabel('Эпоха обучения')
    plt.ylabel('Ошибка')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
ep = [60, 35, 30, 16, 25]

In [ ]:
seeds = [1] * 5
recalls = [0.5]*5
models = [None]*5
seeds

In [ ]:
seed(2)
tf.random.set_seed(3)
d = 0
model = build_model(th=y_data[d].mean())
history = model.fit(x_train, 
                    y_train[d], 
                    batch_size=len(x_train), 
                    epochs=ep[d],
                    validation_data=(x_val, y_val[d]),
                    verbose=0)

plot(history)

In [ ]:
%%time
#Updated, i=14 recall=0.7714
d = 0
for i in range(50):
    seed = i + seeds[d]
    tf.random.set_seed(seed)
    model = build_model()
    history = model.fit(x_train, y_train[d], batch_size=len(x_train), epochs=ep[d], validation_data=(x_val, y_val[d]), verbose=0)
    val_probs = model.predict(x_val)
    th, recall = best_th(y_val[d], val_probs)
    if recall > recalls[d]:
        recalls[d] = recall
        models[d] = model
        seeds[d] = seed
        print("Updated, seed=%d recall=%.4f (d=%d)" % (seed, recall, d))
print("d=%d Best Recall %.4f" % (d, recalls[d]))        

In [ ]:
inds = [50]*50

In [ ]:
%%time
for d in range(5):
    for i in range(inds[d], inds[d]+50):
        seed = i + seeds[d]
        tf.random.set_seed(seed)
        model = build_model()
        history = model.fit(x_train, y_train[d], batch_size=len(x_train), epochs=ep[d], validation_data=(x_val, y_val[d]), verbose=0)
        val_probs = model.predict(x_val)
        th, recall = best_th(y_val[d], val_probs)
        if recall > recalls[d]:
            recalls[d] = recall
            models[d] = model
            seeds[d] = seed
            print("Updated, seed=%d recall=%.4f (d=%d)" % (seed, recall, d))
    print("d=%d Best Recall %.4f" % (d, recalls[d]))    
    inds[d] += 50    

```
([0.7730158730158729,
  0.6905405405405406,
  0.6694117647058824,
  0.6675479180436219,
  0.5775091960063059],
 [63, 32, 39, 42, 128],
 ```

In [ ]:
recalls, seeds, models

In [ ]:
ths = [0.5]*5
y_preds = [None] * 5
avg_recall = 0.
for d in range(5):
    val_probs = models[d].predict(x_val)
    th, recall = best_th(y_val[d], val_probs)
    test_probs = models[d].predict(x_test)
    y_preds[d] = get_pred(test_probs, th)
    avg_recall += recall
    ths[d] = th
    print("d=%d th=%.3f Recall=%.4f" % (d, th, recall))
avg_recall /= 5
print("Avg Recall=%.4f" % avg_recall)


In [ ]:
res_df = pd.DataFrame()
res_df['ID'] = test_df.ID.values
for d in range(5):
    res_df[diseases[d]] = y_preds[d]
res_df


In [ ]:
name = 'submit-%4d.csv' % int(avg_recall * 1e4)
print(name)
res_df.to_csv(name, index=False)

## Fit threshold

In [ ]:
sds = [63, 103, 126, 112, 277]
for d in range(5):
    seed = sds[d]
    tf.random.set_seed(seed)
    model = build_model()
    history = model.fit(x_train, y_train[d], batch_size=len(x_train), epochs=ep[d], validation_data=(x_val, y_val[d]), verbose=0)
    val_probs = model.predict(x_val)
    th, recall = best_th(y_val[d], val_probs)
    print("seed=%d recall=%.4f (d=%d)" % (seed, recall, d))

In [ ]:
from hyperopt import fmin, tpe, hp
best = fmin(fn=lambda x: x ** 2,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100)
print(best)

In [ ]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(x):
    return {
        'loss': x ** 2,
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time': time.time(),
        'other_stuff': {'type': None, 'value': [0, 1, 2]},
        # -- attachments are handled differently
        'attachments':
            {'time_module': pickle.dumps(time.time)}
        }
trials = Trials()
best = fmin(objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100,
    trials=trials)

print(best)

In [ ]:
best

In [ ]:
objective(best['x'])

In [ ]:
msg = trials.trial_attachments(trials.trials[5])['time_module']
time_module = pickle.loads(msg)

In [ ]:
from hyperopt import hp
import hyperopt.pyll.stochastic

space = hp.choice('a',
    [
        ('case 1', 1 + hp.lognormal('c1', 0, 1)),
        ('case 2', hp.uniform('c2', -10, 10))
    ])
print(hyperopt.pyll.stochastic.sample(space))